### Basic Sentiment Analysis

Import the relevant modules:

In [3]:
from textblob import TextBlob
from wordcloud import WordCloud
# from tqdm import tqdm as prog_bar
import pandas as pd
import numpy as np
import tweepy
import re
import matplotlib.pyplot as plt
from tokens import api_keys
plt.style.use('fivethirtyeight')

In [4]:
# twitter authentication code, to connect to the API
authentication = tweepy.OAuthHandler(api_keys["consumer_key"], api_keys["consumer_secret"])
# set tokens
authentication.set_access_token(api_keys["access_token"], api_keys["access_secret"])
# create API object
api = tweepy.API(authentication, wait_on_rate_limit=True)

Extract data

In [5]:
tweets = api.user_timeline(screen_name="Nigel_Farage", count=15700, lang="en", tweet_mode="extended")

# test to see if the API is working

print("Showing last 10 tweets")
i = 1
for each_tweet in tweets[:10]:
    print(f"{str(i)}: {each_tweet.full_text}\n")
    i += 1

Showing last 10 tweets
1: 12 o’clock, first customer in. Love it. https://t.co/pBYm9rtejA

2: RT @Telegraph: Number of high-profile organisations are forced to distance themselves from Black Lives Matter as they say organisation has…

3: Now many others have woken up, this feels like a good time to share this video again. https://t.co/OV544J0ZGF

4: A welcome outbreak of common sense from the Premier League. 

https://t.co/Y1Q9ObtVJv

5: Large scale infrastructure spending was first proposed by The Brexit Party. Not only does Boris now use our policies he’s even using our colours too! https://t.co/SZRLUj5aQs

6: Is Starmer now more Conservative than Boris, who does not have the moral courage to condemn the anti-police marxists?

7: Heartily agree with @Keir_Starmer’s condemnation of the Black Lives Matter organisation. https://t.co/LSwoIhoWk1

8: RT @talkRADIO: Brexit Party leader Nigel Farage will be joining Julia from 8am. Watch live ► https://t.co/GUMvcJzWzS

@JuliaHB1 | @Nigel_Fa…


In [ ]:
data_frame = pd.DataFrame([each_tweet.full_text for each_tweet in tweets], columns="Tweets")